Step 1: Import Libraries

Start by importing the necessary libraries for data manipulation, visualization, natural language processing and machine learning.

In [ ]:
# Import Libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import re
import nltk
from wordcloud import WordCloud
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer

Step 2: Load and Explore Data

Load your dataset containing house features and prices using Pandas and explore it through Exploratory Data Analysis (EDA).

In [ ]:
# Load Dataset
df = pd.read_csv('amazon_alexa.tsv', delimiter='\t')

In [ ]:
# Display the First few Rows of Dataset
df.head()

In [ ]:
# Check Missing Values
df.isnull().sum()

In [ ]:
# Get Information about the Dataset, Missing Values and Datatypes
df.info()

Step 3: Data Preprocessing and Feature Engineering

Based on the insights from EDA, perform data preprocessing and feature engineering to prepare the data for Random Forest Classifier.

In [ ]:
# value counts of variation 
variation_label = df.variation.value_counts()[:5]
print(variation_label)
index=variation_label.index
print(index)

# plotting top 5 variation
plt.figure(figsize=(10, 5))
sns.barplot(y=variation_label,x=index);
plt.xlabel('Variation')
plt.ylabel('Count')
plt.title('Top 5 Variation', fontsize=20);

In [ ]:
# shape of data
df.shape

In [ ]:
# adding a len column for analyzing the length of the reviews
df['len'] = df['verified_reviews'].map(len)
df['len']

In [ ]:
# displaying data based on len
df.groupby('len').describe().head(10)

In [ ]:
# displaying the data according to the Ratings
df.groupby('rating').describe()

In [ ]:
# displaying data based according to the feedback
df.groupby('feedback').describe()

In [ ]:
# visualization

# value counts of rating
rating_label = df.rating.value_counts()
print(rating_label)
index=rating_label.index

# barplot
plt.figure(figsize=(10, 5))
sns.barplot(x=index, y=rating_label )
plt.xlabel('Rating')
plt.ylabel('Count')
plt.title('Bar Plot of Ratings', fontsize=20)

In [ ]:
# histogram plot of len of reivews

len_label = df['len'].value_counts()
plt.figure(figsize=(10, 5))
sns.histplot(len_label, bins=100 ,kde=True);
plt.xlabel('Length', fontsize=20)
plt.ylabel('Count', fontsize=20);
plt.title('Distribution of Lenght in Revies', fontsize=20);

In [ ]:
# Let's Check some of the reviews according to thier lengths
df[df.len == 1]['verified_reviews'].iloc[0]

In [ ]:
df[df['len'] == 150]['verified_reviews'].iloc[0]

In [ ]:
df[df['len'] == 50]['verified_reviews'].iloc[0]

In [ ]:
df[df['len'] == 25]['verified_reviews'].iloc[0]

In [ ]:
# box plot 

plt.figure(figsize=(10, 5))
sns.boxplot(x=df['rating'],y= df['len'], palette = 'Blues')
plt.xlabel('Rating')
plt.ylabel('len')
plt.title('Length vs Ratings', fontsize=20)

In [ ]:
# violin plot
plt.figure(figsize=(10, 5))
sns.violinplot(x=df['feedback'],y= df['rating'], palette = 'cool')
plt.xlabel('Feedback')
plt.ylabel('Rating')
plt.title('Feedback vs Ratings', fontsize=20)

In [ ]:
# swarmplot

plt.figure(figsize=(10, 5))
sns.swarmplot(x=df['variation'], y=df['len'], palette = 'deep')
plt.xlabel('Variation')
plt.ylabel('len')
plt.xticks(rotation = 75)
plt.title("Variation vs Length of Ratings", fontsize=20)


In [ ]:
# bivariate analysis 

plt.figure(figsize=(10, 5))
sns.boxenplot(x=df['variation'], y=df['rating'], palette = 'spring')
plt.xlabel('Variation')
plt.ylabel('Rating')
plt.xticks(rotation = 70)
plt.title("Variation vs Ratings", fontsize=20)

In [ ]:
# preprocessing of text 

# CountVectorrizer 
count_vector = CountVectorizer(stop_words='english') 

ws = count_vector.fit_transform(df.verified_reviews)
s_w = ws.sum(axis=0)
w_f = [(w, s_w[0, idx]) for w, idx in count_vector.vocabulary_.items()]
w_f = sorted(w_f, key = lambda x: x[1], reverse = True)

# creating dataframe
freq = pd.DataFrame(w_f, columns=['word', 'freq'])

In [ ]:
# barplot of top 50 frequently occuring words
color = plt.cm.ocean(np.linspace(0, 1, 20))
freq.head(50).plot(x='word', y='freq', kind='bar', figsize=(15, 6), color = color)
plt.xlabel('word')
plt.ylabel('length')
plt.title('Most Frequently Occuring Words - Top 50',fontsize=20)

In [ ]:
# representing words on WordCloud
word_cloud = WordCloud(background_color='black', width=1500, height=1400).generate_from_frequencies(dict(w_f))
plt.figure(figsize=(10, 10))
plt.axis('off')
plt.imshow(word_cloud)
plt.title('Vocabulary from Reviews',fontsize=20)

In [ ]:
# preproecessing
c = []

for i in range(0, 3150):
    # removing characters except a-z and A-Z
    r = re.sub('[^a-zA-Z]', ' ', df['verified_reviews'][i])
    # converting every word into lower word
    r = r.lower()
    # splitting text 
    r = r.split()
    # applying Stemming
    ps = PorterStemmer()
    # removing stopwords
    sw = stopwords.words('english')
    sw.remove('not')
    r = [ps.stem(word) for word in r if not word in set(sw)]
    r = ' '.join(r)
    c.append(r)

In [ ]:
# count vectorizer
count_vector = CountVectorizer(max_features=2500) 

# independent and dependent variables
X = count_vector.fit_transform(c).toarray()
y = df.iloc[:, 4].values

# checking shape 
print(X.shape)
print(y.shape)

In [ ]:
# train test split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
print("shape of X_train: ", X_train.shape)
print("shape of X_test: ", X_test.shape)

In [ ]:
# Min Max Scaler 
min_max_sc = MinMaxScaler()

X_train = min_max_sc.fit_transform(X_train)
X_test = min_max_sc.transform(X_test)

Step 4: Create and Train the Random Forest Classifier Model

Create a Random Forest Classifier model using scikit-learn.

In [ ]:
# Random Forest Classifier 
rf = RandomForestClassifier()

# fit on data 
rf.fit(X_train, y_train)

# prediction 
y_pred = rf.predict(X_test)


Step 6: Evaluate Model Performance

Assess the model's performance using evaluation metrics like Accuracy, Confusion Matrix, Classification Report

In [ ]:
# Accuracy Score
accuracy=accuracy_score(y_test,y_pred)
print('Accuracy Score For The Model=',accuracy)

In [ ]:
# Classification Report
report=classification_report(y_test,y_pred)
print('Classification Report',report)

In [ ]:
# confusion matrix 
plt.figure(figsize=(10, 6))
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='g')
plt.title('Confusion Matrix')

Step 7: HyperParameter Tunning

Hyperparameter tuning is the process of finding the best combination of hyperparameters for a machine learning model to optimize its performance.

In [ ]:
# Hyperparameter Tunning

p = {
    'bootstrap': [True],
    'max_depth': [80, 100],
    'min_samples_split': [8, 12],
    'n_estimators': [100, 300]
}

# Grid Search CV 
grid_cv = GridSearchCV(estimator=rf, param_grid=p, cv=10, verbose=0, scoring='accuracy', n_jobs=-1, 
                       return_train_score=True)
grid_cv.fit(X_train, y_train.ravel())

In [ ]:
print("Best Parameter : {}".format(grid_cv.best_params_))

In [ ]:
rf_cv = RandomForestClassifier(bootstrap=True, max_depth=80, min_samples_split=8, n_estimators=300)

# fit on data 
rf_cv.fit(X_train, y_train)

# prediction 
pred = rf_cv.predict(X_test)

In [ ]:
# Accuracy Score
accuracy=accuracy_score(y_test,pred)
print('Accuracy Score For The Model=',accuracy)

In [ ]:
# Classification Report
report=classification_report(y_test,pred)
print('Classification Report',report)

In [ ]:
# confusion matrix 
plt.figure(figsize=(10, 6))
sns.heatmap(confusion_matrix(y_test, pred), annot=True, fmt='g')
plt.title('Confusion Matrix', fontsize=20)